## YouTube Ranking

In [2]:
import requests, time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

- Step 1, 2

In [3]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [4]:
trs = soup.select('tr.aos-init')
len(trs)

0

In [5]:
driver = webdriver.Chrome()
driver.get(url)

In [6]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('tr.aos-init')
len(trs)

100

- Step 3

In [7]:
tr = trs[0]
rank = tr.select_one('.rank').get_text().strip()
rank

'1'

In [10]:
category = tr.select_one('.category').get_text().strip()[1:-1]        # [음악/댄스/가수]
category

'음악/댄스/가수'

In [11]:
channel = tr.select_one('.subject > h1 > a').get_text().strip()
channel

'BLACKPINK'

In [12]:
subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
subscriber

'9130만'

In [31]:
view = tr.select_one('.view_cnt').get_text().strip()
video = tr.select_one('.video_cnt').get_text().strip()
view, video

('213억4892만', '2,330개')

In [32]:
# 숫자로 바꿔주는 함수 / '비공' 존재
def convert(s):
    s = s.replace('억','').replace('개', '').replace(',', '').replace('만', '0000')
    return int(s)

In [33]:
convert(view)

21348920000

- Step 4

In [34]:
data = []
for tr in trs:
    rank = int(tr.select_one('.rank').get_text().strip())
    category = tr.select_one('.category').get_text().strip()[1:-1]
    channel = tr.select_one('.subject > h1 > a').get_text().strip()
    subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
    view = convert(tr.select_one('.view_cnt').get_text().strip())
    video = convert(tr.select_one('.video_cnt').get_text().strip())
    data.append({'순위': rank, '채널명': channel, '카테고리': category,
                 '구독자수': subscriber, '조회수': view, '비디오수': video})

- Step 5. 에러 수정

In [21]:
try:
    for index, tr in enumerate(trs):
        rank = int(tr.select_one('.rank').get_text().strip())
        category = tr.select_one('.category').get_text().strip()[1:-1]
        channel = tr.select_one('.subject > h1 > a').get_text().strip()
        subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
        view = convert(tr.select_one('.view_cnt').get_text().strip())
        video = convert(tr.select_one('.video_cnt').get_text().strip())
except:
    print(index + 1)        # 순위가 출력되게

2


In [35]:
data = []
for tr in trs:
    rank = int(tr.select_one('.rank').get_text().strip())
    category = tr.select_one('.category').get_text().strip()[1:-1]
    channel = tr.select_one('.subject > h1 > a').get_text().strip()
    subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
    view = convert(tr.select_one('.view_cnt').get_text().strip())
    video = convert(tr.select_one('.video_cnt').get_text().strip())
    data.append({'순위': rank, '채널명': channel, '카테고리': category,
                 '구독자수': subscriber, '조회수': view, '비디오수': video})

- Step 6

In [37]:
data = []
for page in range(1, 11):
    url = f'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page={page}'
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.select('tr.aos-init')

    for tr in trs:
        rank = int(tr.select_one('.rank').get_text().strip())
        category = tr.select_one('.category').get_text().strip()[1:-1]
        channel = tr.select_one('.subject > h1 > a').get_text().strip()
        subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
        view = convert(tr.select_one('.view_cnt').get_text().strip())
        video = convert(tr.select_one('.video_cnt').get_text().strip())
        data.append({'순위': rank, '채널명': channel, '카테고리': category,
                    '구독자수': subscriber, '조회수': view, '비디오수': video})
    
driver.close()

- Step 7

In [39]:
df = pd.DataFrame(data)
df.head()

,순위,채널명,카테고리,구독자수,조회수,비디오수
0,1,BLACKPINK,음악/댄스/가수,91300000,33206830000,567
1,2,BANGTANTV,음악/댄스/가수,76400000,21348920000,2330
2,3,HYBE LABELS,음악/댄스/가수,72700000,29979900000,1428
3,4,SMTOWN,음악/댄스/가수,32100000,27856680000,4227
4,5,JYP Entertainment,음악/댄스/가수,27800000,20510660000,1833


In [41]:
df.to_csv('data/유튜브랭킹_20230919.csv', index=False)
pd.read_csv('data/유튜브랭킹_20230919.csv').head()

,순위,채널명,카테고리,구독자수,조회수,비디오수
0,1,BLACKPINK,음악/댄스/가수,91300000,33206830000,567
1,2,BANGTANTV,음악/댄스/가수,76400000,21348920000,2330
2,3,HYBE LABELS,음악/댄스/가수,72700000,29979900000,1428
3,4,SMTOWN,음악/댄스/가수,32100000,27856680000,4227
4,5,JYP Entertainment,음악/댄스/가수,27800000,20510660000,1833
